# Affinity Propogation Algorithm from Scratch

In [100]:
import numpy as np
import pandas as pd

### Generating the Dataset

In [101]:
d = {}
d['Participent'] = ['Alice','Bob','Cary','Doug','Edna']
d['Tax Rate'] = [3,4,3,2,1]
d['Fee'] = [4,3,5,1,1]
d['Interest Rate'] = [3,5,3,3,3]
d['Quantity Limit'] = [2,1,3,3,2]
d['Price Limit'] = [1,1,3,2,3]
df = pd.DataFrame(d)

In [102]:
df

,Participent,Tax Rate,Fee,Interest Rate,Quantity Limit,Price Limit
0,Alice,3,4,3,2,1
1,Bob,4,3,5,1,1
2,Cary,3,5,3,3,3
3,Doug,2,1,3,3,2
4,Edna,1,1,3,2,3


### Extracting the data as points and creating a numpy array

In [103]:
X = np.array([df.iloc[i,1:].values for i in range(df.shape[0])])

In [104]:
X

array([[3, 4, 3, 2, 1],
       [4, 3, 5, 1, 1],
       [3, 5, 3, 3, 3],
       [2, 1, 3, 3, 2],
       [1, 1, 3, 2, 3]], dtype=object)

### Set the parameters here

In [105]:
damping = 0.8 #Extra Noise to assure convergence
n = 300 #Number of iterations

### Create matrices with default value

In [106]:
s_mat = np.zeros(X.shape) #Similarity Matrix
r_mat = np.zeros(X.shape) #Responsibility Matrix
a_mat = np.zeros(X.shape) #Availability Matrix
c_mat = np.zeros(X.shape) #Criterion Matrix 

In [107]:
def similarity_matrix():
    ''' It will create/update and return the similarity matrix'''
    min=np.inf
    for i in range(X.shape[0]):
        for k in range(X.shape[0]):
            s_mat[i,k] = -sum((X[i]-X[k])**2)
            if(s_mat[i,k]<min):
                min = s_mat[i,k]
    for i in range(X.shape[0]):
        s_mat[i,i]=min
    return s_mat
    

In [108]:
def responsibility_matrix():
    ''' It will create/update and return the responsibility matrix'''
    for i in range(X.shape[0]):
        for k in range(X.shape[0]):
            r_mat[i,k] = s_mat[i,k] - max((np.delete(s_mat[i,:],k))+np.delete(a_mat[i,:],k))
    return r_mat

In [109]:
def availability_matrix():
    ''' It will create/update and return the availability matrix'''
    for i in range(X.shape[0]):
        for k in range(X.shape[0]):
            m=np.delete(r_mat[:,k],i)
            if(i==k):
                a_mat[i,k]=sum(m[m>0])
            else:
                if(r_mat[k,k]+sum(m[m>0])>0):
                    a_mat[i,k]=0
                else:
                    a_mat[i,k]=r_mat[k,k]+sum(m[m>0])
    return a_mat

### Final Calculation to create Criterion Matrix

In [110]:
for i in range(n):
    similarity_matrix()
    if i>0:
        r_mat = damping*r_mat+(1-damping)*responsibility_matrix()
        a_mat = damping*a_mat+(1-damping)*availability_matrix()
    else: #To execute it for first time 
        responsibility_matrix()
        availability_matrix()
c_mat = r_mat + a_mat #Criterion Matrix 

### Extracting the cluster from Criterion Matrix

In [111]:
clusters = [np.argmax(c_mat[i]) for i in range(X.shape[0])] #Finding the cluster based on position of max value in a row

### Final Clusters

In [112]:
print(clusters)

[0, 0, 0, 3, 3]
